**image to Json conversion with Transformer**

In [1]:
!python -m pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-zwj0vxwr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-zwj0vxwr
  Resolved https://github.com/huggingface/transformers to commit a7f29523361b2cc12e51c1f5133d95f122f6f45c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 10.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11106589 sha256=0f1a7ec09a1c80a444a9552566bb1447e3f3993bf3aa1377cecf0c31570e1f6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ghktjb0l/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
 

In [2]:
!pip install -q gradio

In [3]:
import gradio as gr
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import torch
import requests

In [4]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

...

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
def extract_invoice_data(image):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                },
                {
                    "type": "text",
                    "text": """You are an expert in invoice data extraction.
                Extract data from the provided invoice images, based on the individual items and
                format the output as JSON."""
                }
            ]
        }
    ]

    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt"
    )

    # Move inputs to GPU if available
    inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")

    output_ids = model.generate(**inputs, max_new_tokens=1024)

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    return output_text[0]

In [12]:
text=extract_invoice_data("BankStatementChequing_1.png")
print(text)

```json
[
  {
    "Date": "2003-10-08",
    "Description": "Previous balance",
    "Ref": "0.55",
    "Withdrawals": "694.81",
    "Deposits": "695.36",
    "Balance": "1,515.63"
  },
  {
    "Date": "2003-10-14",
    "Description": "Payroll Deposit - HOTEL",
    "Ref": "694.81",
    "Withdrawals": "695.36",
    "Deposits": "0.00",
    "Balance": "1,515.63"
  },
  {
    "Date": "2003-10-14",
    "Description": "Web Bill Payment - MASTERCARD",
    "Ref": "9685",
    "Withdrawals": "200.00",
    "Deposits": "495.36",
    "Balance": "1,415.63"
  },
  {
    "Date": "2003-10-16",
    "Description": "ATM Withdrawal - INTERAC",
    "Ref": "3990",
    "Withdrawals": "21.25",
    "Deposits": "474.11",
    "Balance": "1,436.88"
  },
  {
    "Date": "2003-10-16",
    "Description": "Fees - Interac",
    "Ref": "1.50",
    "Withdrawals": "472.61",
    "Deposits": "0.00",
    "Balance": "1,436.88"
  },
  {
    "Date": "2003-10-20",
    "Description": "Interac Purchase - ELECTRONICS",
    "Ref": "19

In [14]:
text=extract_invoice_data("BankStatementChequing.png")
print(text)

```json
[
  {
    "date": "2003-10-09",
    "description": "Payroll Deposit - HOTEL",
    "amount": 694.81
  },
  {
    "date": "2003-10-14",
    "description": "Web Bill Payment - MASTERCARD",
    "amount": 9685.20
  },
  {
    "date": "2003-10-14",
    "description": "Interac Transaction - INTERAC",
    "amount": 1.50
  },
  {
    "date": "2003-10-21",
    "description": "Web Bill Payment - AMEX",
    "amount": 334.00
  },
  {
    "date": "2003-10-21",
    "description": "Interac Transaction - INTERAC",
    "amount": 29.08
  },
  {
    "date": "2003-10-21",
    "description": "Interac Transaction - INTERAC",
    "amount": 197.29
  },
  {
    "date": "2003-10-21",
    "description": "Interac Transaction - INTERAC",
    "amount": 197.29
  },
  {
    "date": "2003-10-30",
    "description": "Web Funds Transfer - FROM SAVINGS",
    "amount": 2620.00
  },
  {
    "date": "2003-11-03",
    "description": "Cheque No. 400",
    "amount": 100.00
  },
  {
    "date": "2003-11-03",
    "descrip

Next steps

Vision Transformer (ViT)
https://pub.aimind.so/how-to-extract-the-data-from-the-invoice-image-in-json-form-2ea1ac4ecdc1